In [52]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import operator
from collections import Counter
from operator import itemgetter
import time
import string

In [2]:
%%time
df_small = pd.read_csv('../data/small_sample.csv')
df = pd.read_csv('../data/large_sample.csv')
df_small.drop(columns=['Unnamed: 0'], inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
df.rating = np.ceil(df.rating)
df_small.rating = np.ceil(df_small.rating)

In [3]:
def create_count(comment):
    for x in comment.split(' '):
        if x in di:
            di[x.lower()] += 1
        else:
            di[x.lower()] = 1
    return di

In [4]:
%%time
di = dict()
df.loc[df.rating == 1].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
one_dict = di
di = dict()
df.loc[df.rating == 2].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
two_dict = di
di = dict()
df.loc[df.rating == 3].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
three_dict = di
di = dict()
df.loc[df.rating == 4].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
four_dict = di
di = dict()
df.loc[df.rating == 5].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
five_dict = di
di = dict()
df.loc[df.rating == 6].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
six_dict = di
di = dict()
df.loc[df.rating == 7].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
seven_dict = di
di = dict()
df.loc[df.rating == 8].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
eight_dict = di
di = dict()
df.loc[df.rating == 9].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
nine_dict = di
di = dict()
df.loc[df.rating == 10].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
ten_dict = di
time = '37s'

CPU times: user 36.9 s, sys: 353 ms, total: 37.2 s
Wall time: 37.4 s


In [5]:
%%time
one_counter = Counter(one_dict)
two_counter = Counter(two_dict)
three_counter = Counter(three_dict)
four_counter = Counter(four_dict)
five_counter = Counter(five_dict)
six_counter = Counter(six_dict)
seven_counter = Counter(seven_dict)
eight_counter = Counter(eight_dict)
nine_counter = Counter(nine_dict)
ten_counter = Counter(ten_dict)

CPU times: user 49.1 ms, sys: 8.91 ms, total: 58.1 ms
Wall time: 62.8 ms


In [6]:
all_counter = one_counter + two_counter + three_counter + four_counter + five_counter + six_counter + seven_counter + eight_counter + nine_counter + ten_counter

In [7]:
all_dict = dict(Counter(all_counter))

In [8]:
%%time
new_data = {k: v for k, v in all_dict.items() if v > 5}

CPU times: user 22.2 ms, sys: 1.67 ms, total: 23.9 ms
Wall time: 23 ms


In [9]:
%%time
sortedList=sorted(new_data.values())
sorted_dict = dict()
for sortedKey in sortedList:
    for key, value in new_data.items():
        if value==sortedKey:
            sorted_dict[key]=value
            
time = '1min 32sec'

CPU times: user 1min 31s, sys: 204 ms, total: 1min 31s
Wall time: 1min 32s


In [10]:
%%time
dict_list = []
for key, value in sorted_dict.items():
    temp = [key,value]
    dict_list.append(temp)

CPU times: user 44.7 ms, sys: 12.8 ms, total: 57.5 ms
Wall time: 56.8 ms


In [11]:
testing_df = df.loc[df.comment.apply(lambda x: len(x.split(' ')))<30].copy()

In [12]:
def make_it_lower(comment):
    new_comment = []
    for x in comment.split(' '):
        new_comment.append(x.lower())
    return ' '.join(new_comment)

In [13]:
%%time
testing_df['lower_comment'] = testing_df.comment.apply(lambda x: make_it_lower(x))
testing_df.drop(columns=['comment'], inplace=True)
testing_df.rename(columns={'lower_comment':'comment'}, inplace=True)

CPU times: user 2.89 s, sys: 70.1 ms, total: 2.96 s
Wall time: 2.98 s


In [14]:
def word_count(comment, word, number):
    if word in comment.split(' '):
        if word in number:
            number[word] += comment.split(' ').count(word)
        else:
            number[word] = comment.split(' ').count(word)

In [33]:
%%time
def make_report(w, z, dictionary, dataframe):
    dict_list = dictionary
    testing_df = dataframe.copy()
    labels = ['word', 'average/std', 'percent_average', 'percent_std', 'rating', 'word_count', 'divisor', 'percent', 'time']
    to_return = pd.DataFrame(columns=labels)
    for word in dict_list[int(w):int(z)]:
        start_time = time.time()
        to_append = []
        std = 0
        average = 0
        for y in range(1,11):
            number = dict()
            testing_df.loc[testing_df.rating == y].comment.apply(word_count, word=word[0], number=number)
            divisor = testing_df.loc[df.rating == y].comment.count()
            if word[0] in number:
                percent = number[word[0]]/divisor
            else:
                percent = 0
                number[word[0]] = 0
            to_append.append([word[0], None, None, None, y, number[word[0]], divisor, percent, None])
        for_array = []
        for value in to_append:
            for_array.append(value[7])
        x = np.array(for_array)
        array1 = [[word[0], (np.sum(x)/10)/(np.std(x)), np.sum(x)/10, np.std(x), None, None, None, None, time.time()-start_time]]
        for x in to_append:
            array1.append(x)
        inbetween_df = pd.DataFrame.from_records(array1, columns=labels)
        to_return = pd.concat([to_return, inbetween_df])
    return to_return

CPU times: user 9 µs, sys: 1 µs, total: 10 µs
Wall time: 14.8 µs


In [34]:
%%time
report = make_report(0,100, dict_list, testing_df)

CPU times: user 2min 32s, sys: 3.31 s, total: 2min 36s
Wall time: 2min 37s


In [37]:
report

,word,average/std,percent_average,percent_std,rating,word_count,divisor,percent,time
0,denominator,0.380951,0.000051,0.000134,NaN,NaN,NaN,NaN,1.714518
1,denominator,NaN,NaN,NaN,1.0,1.0,2219.0,0.000451,NaN
2,denominator,NaN,NaN,NaN,2.0,0.0,4575.0,0.000000,NaN
3,denominator,NaN,NaN,NaN,3.0,0.0,8644.0,0.000000,NaN
4,denominator,NaN,NaN,NaN,4.0,0.0,15691.0,0.000000,NaN
5,denominator,NaN,NaN,NaN,5.0,0.0,29252.0,0.000000,NaN
6,denominator,NaN,NaN,NaN,6.0,2.0,61498.0,0.000033,NaN
7,denominator,NaN,NaN,NaN,7.0,3.0,113929.0,0.000026,NaN
8,denominator,NaN,NaN,NaN,8.0,0.0,126791.0,0.000000,NaN
9,denominator,NaN,NaN,NaN,9.0,0.0,71451.0,0.000000,NaN


In [106]:
report.to_csv('report for {}.csv'.format('-10--5'))

In [107]:
df = pd.read_csv('report for -10--5')

In [108]:
len(dict_list)

22543

In [118]:
dict_list[22540:22542]

[['2', 12685], ['-', 17661]]

In [119]:
report

,word,average/std,percent_average,percent_std,rating,word_count,divisor,percent,time


960.0

In [45]:
def write_csv(low, high, df):
    df.to_csv('report_for_{}_to_{}.csv'.format(low, high))

In [31]:
write_csv(0,100,report)

In [47]:
def loop_write_report(dictionary, dataframe, number, count):
    num = int(number)
    size = int(count)
    first = 0
    while first < num - 1:
        if first + size < num - 2:
            report = make_report(first, first + size, dictionary, dataframe)
            write_csv(first, first + size, report)
            print('dataframe written from {} to {}'.format(first, first + size))
            first = first + size + 1
        else:
            report = make_report(first, num - 1, dictionary, dataframe)
            write_csv(first, num - 1, report)
            print('dataframe written from {} to {}'.format(first, num - 1))
            print('last dataframe')
            break

In [56]:
loop_write_report(dictionary=dict_list, dataframe=testing_df, number=len(dict_list), count=200)

dataframe written from 0 to 200
dataframe written from 201 to 401
dataframe written from 402 to 602
dataframe written from 603 to 803
dataframe written from 804 to 1004
dataframe written from 1005 to 1205
dataframe written from 1206 to 1406
dataframe written from 1407 to 1607
dataframe written from 1608 to 1808
dataframe written from 1809 to 2009
dataframe written from 2010 to 2210
dataframe written from 2211 to 2411
dataframe written from 2412 to 2612
dataframe written from 2613 to 2813
dataframe written from 2814 to 3014
dataframe written from 3015 to 3215
dataframe written from 3216 to 3416
dataframe written from 3417 to 3617
dataframe written from 3618 to 3818
dataframe written from 3819 to 4019
dataframe written from 4020 to 4220
dataframe written from 4221 to 4421
dataframe written from 4422 to 4622
dataframe written from 4623 to 4823
dataframe written from 4824 to 5024
dataframe written from 5025 to 5225
dataframe written from 5226 to 5426
dataframe written from 5427 to 5627
dat

KeyboardInterrupt: 

In [41]:
type(testing_df)

pandas.core.frame.DataFrame

In [55]:
len(dict_list)

22543

In [48]:
s.translate(None, string.punctuation)

NameError: name 's' is not defined

In [49]:
phrase = "hello! I wanna remove *@ .,/ if possible"

In [53]:
phrase.translate(None, string.punctuation)

TypeError: translate() takes exactly one argument (2 given)

In [54]:
table = str.maketrans(dict.fromkeys('!@#$%^&*()_+=~`[{}]|\:;"<,>.?/'))
phrase.translate(table)

'hello I wanna remove   if possible'